<a href="https://colab.research.google.com/github/TesisDeepcodeUcuenca/Tesis-Deepcode-Ucuenca/blob/main/Experimentos/Sin%20Feedback/Experimento_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Deepcode: 2 Capas lineales Encoder 1 entradas, salidas 3 -> 2 Capas Lineales Decoder 3 entradas, 1 salida sin FEEDBACK, rate 1/3
__author__ = 'yihanjiang'
# Adding the *Receiver Encoding*
import argparse
import math
import random
import torch
import torch.optim as optim
import numpy as np
import torch.nn.functional as F
from tqdm import tqdm
import matplotlib.pyplot as plt
from google.colab import files
import csv
import locale
import pandas as pd

def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('-init_nw_weight', type=str, default='default')
    parser.add_argument('-code_rate', type=int, default=3)
    parser.add_argument('-learning_rate', type = float, default=0.00085)
    parser.add_argument('-batch_size', type=int, default=100)
    parser.add_argument('-num_epoch', type=int, default=10)
    parser.add_argument('--no-cuda', action='store_true', default=False, help='disables CUDA training')
    parser.add_argument('-block_len', type=int, default=50)
    parser.add_argument('-num_block', type=int, default=100000)
    parser.add_argument('-enc_num_layer', type=int, default=2)
    parser.add_argument('-dec_num_layer', type=int, default=2)
    parser.add_argument('-fb_num_layer',  type=int, default=2)
    parser.add_argument('-enc_num_unit',  type=int, default=100)
    parser.add_argument('-dec_num_unit',  type=int, default=100)
    parser.add_argument('-fb_num_unit',   type=int, default=100)
    parser.add_argument('-q_train', type=float, default=0.001, help='Valor de probabilidad q de entrenamiento')
    parser.add_argument('-q_train_fb', type=float, default=0.0000001, help='Valor de probabilidad q de entrenamiento para feedback')
    parser.add_argument('-q_test_start', type=float, default=0.001, help='Inicio de valores de q para pruebas')
    parser.add_argument('-q_test_end', type=float, default=0.15, help='Fin de valores de q para pruebas')
    parser.add_argument('-q_points', type=int, default=5, help='Número de puntos de probabilidad q')
    parser.add_argument('-channel_mode', choices=['normalize', 'lazy_normalize', 'tanh'], default='lazy_normalize')
    parser.add_argument('-enc_act', choices=['tanh', 'selu', 'relu', 'elu', 'sigmoid'], default='sigmoid')
    parser.add_argument('--zero_padding', action='store_true', default=False, help='enable zero padding')
    parser.add_argument("-f", "--file", required=False)
    args = parser.parse_args()
    return args
#############################
#                           #
#    █████████   ██████████ #
#   ███░░░░░███ ░░███░░░░░█ #
#  ░███    ░███  ░███  █ ░  #
#  ░███████████  ░██████    #
#  ░███░░░░░███  ░███░░█    #
#  ░███    ░███  ░███ ░   █ #
#  █████   █████ ██████████ #
# ░░░░░   ░░░░░ ░░░░░░░░░░  #
#                           #
#############################
class AE(torch.nn.Module):
    def __init__(self, args):
        super(AE, self).__init__()
        self.args             = args
        # Encoder
        self.enc_p2_linear_fwd   = torch.nn.Linear(1, args.enc_num_unit, bias=True)
        self.enc_p2_linear    = torch.nn.Linear(args.enc_num_unit, 3)

        #Decoder
        #self.dec_lstm           = torch.nn.LSTM(args.code_rate,  args.dec_num_unit,
        #                                   num_layers=2, bias=True, batch_first=True,
        #                                   dropout=0, bidirectional=True)
        self.dec_linear        = torch.nn.Linear(args.code_rate, args.dec_num_unit, bias=True)
        self.dec_output        = torch.nn.Linear(args.dec_num_unit, 1)

    def enc_act(self, inputs):
        if self.enc_act == 'tanh':
            return F.tanh(inputs)
        elif self.enc_act == 'elu':
            return F.elu(inputs)
        elif self.enc_act == 'relu':
            return F.relu(inputs)
        elif self.enc_act == 'selu':
            return F.selu(inputs)
        elif self.enc_act == 'sigmoid':
            return F.sigmoid(inputs)
        else:
            return F.tanh(inputs)

    def forward(self, input, fwd_noise, fb_noise):
        x_p1      = input
        x_p1_norm = 2*x_p1-1
        p1_code = x_p1

        if self.args.zero_padding:
            p1_rec  = torch.logical_xor(x_p1, fwd_noise[:,:,0].view(self.args.batch_size, self.args.block_len+1, 1) ).float()
            p1_fb   = torch.logical_xor(p1_rec , fb_noise[:,:, 0].view(self.args.batch_size, self.args.block_len+1, 1) ).float()
        else:
            p1_rec  = torch.logical_xor(x_p1, fwd_noise[:,:,0].view(self.args.batch_size, self.args.block_len, 1) ).float()
            p1_fb   = torch.logical_xor(p1_rec , fb_noise[:,:, 0].view(self.args.batch_size, self.args.block_len, 1) ).float()

        #p1_fb_norm = 2*p1_fb - 1


        for idx in range(input.shape[1]):
            #if idx == 0:
            #    input_tmp        = x_p1_norm[:,idx,:].view(self.args.batch_size, 1, 1)
            #    x_fwd_p2  =        self.enc_act(self.enc_p2_linear_fwd(input_tmp))
            #    x_tmp_p2         = self.enc_act(self.enc_p2_linear(x_fwd_p2))
            #    x_p2_history     = x_tmp_p2

            #else:
            input_tmp        = x_p1_norm[:,idx,:].view(self.args.batch_size, 1, 1)
            x_tmp_p2  =        self.enc_act(self.enc_p2_linear_fwd(input_tmp))
            x_tmp_p2         = self.enc_act(self.enc_p2_linear(x_tmp_p2))
            x_p2_history     = x_tmp_p2

            x_tmp_p2  = torch.heaviside(x_tmp_p2 , torch.zeros(x_tmp_p2.shape, device=x_tmp_p2.device)).float()


            if idx == 0:
              x_tmp_p2_gen= x_tmp_p2

            else:
              x_tmp_p2_gen = torch.cat([x_tmp_p2_gen,x_tmp_p2 ], dim = 1)


            #if idx == 0:
            #    p2_code= x_tmp_p2

            #else:
            #    p2_code = torch.cat([p2_code,x_tmp_p2 ], dim = 1)


            x_p2_rec    = torch.logical_xor(x_tmp_p2,  fwd_noise[:,idx,:].view(self.args.batch_size, 1, 3)).float()
            #x_p2_fb     = torch.logical_xor(x_p2_rec, fb_noise[:,idx, 1:].view(self.args.batch_size, 1, 2)).float()

            #fb_tmp      = 2*x_p2_fb-1

            if idx == 0:
                p2_code= x_tmp_p2
                p2_rec = x_p2_rec
                #p2_fb  = x_p2_fb
            else:
                p2_code = torch.cat([p2_code,x_tmp_p2 ], dim = 1)
                p2_rec = torch.cat([p2_rec,x_p2_rec ], dim = 1)
                #p2_fb  = torch.cat([p2_fb, x_p2_fb],   dim = 1)

        dec_input = p2_rec

        x_code = x_tmp_p2_gen
        inputs_dec = 2*dec_input-1

        #x_dec, hdec_tmp  = self.dec_lstm( inputs_dec.float() )
        x_dec  = self.dec_linear( inputs_dec.float() )
        x_dec     = F.sigmoid(self.dec_output(x_dec))
        return x_dec, x_code
####################
#                  #
#  ██████   ██████ #
# ░░██████ ██████  #
#  ░███░█████░███  #
#  ░███░░███ ░███  #
#  ░███ ░░░  ░███  #
#  ░███      ░███  #
#  █████     █████ #
# ░░░░░     ░░░░░  #
#                  #
####################

args = get_args()
print(args)
def errors_ber(y_true, y_pred):
    if args.zero_padding:
        t1 = torch.round(y_true[:,:-1,:])
        t2 = torch.round(y_pred[:,:-1,:])
    else:
        t1 = torch.round(y_true[:,:,:])
        t2 = torch.round(y_pred[:,:,:])

    myOtherTensor = torch.not_equal(t1, t2).float()
    k = sum(sum(myOtherTensor))/(myOtherTensor.shape[0]*myOtherTensor.shape[1])
    return k
def errors_bler(y_true, y_pred):

    if args.zero_padding:
        t1 = torch.round(y_true[:,:-1,:])
        t2 = torch.round(y_pred[:,:-1,:])
    else:
        t1 = torch.round(y_true[:,:,:])
        t2 = torch.round(y_pred[:,:,:])

    decoded_bits = t1
    X_test       = t2
    tp0 = (abs(decoded_bits-X_test)).reshape([X_test.shape[0],X_test.shape[1]])

    bler_err_rate = sum(torch.sum(tp0,axis=1)>0)*1.0/(X_test.shape[0])
    return bler_err_rate

def ber_pos(y_true, y_pred):
    if args.zero_padding:
        t1 = torch.round(y_true[:,:-1,:])
        t2 = torch.round(y_pred[:,:-1,:])
    else:
        t1 = torch.round(y_true[:,:,:])
        t2 = torch.round(y_pred[:,:,:])
    dif = torch.not_equal(t1, t2).float()
    suma_dif = (torch.sum(dif, dim=0))
    promedio_errorxbit = suma_dif/(dif.shape[0])
    return promedio_errorxbit

def Guardar_resultados(BER_FINAL, args, model):
    Q = [BER_FINAL[i] for i in range(len(BER_FINAL)) if i % 2 == 0]
    ber = [BER_FINAL[i] for i in range(len(BER_FINAL)) if i % 2 != 0]
    column_names = ['init_nw_weight', 'code_rate', 'learning_rate', 'batch_size', 'num_epoch', 'no_cuda',
                    'block_len', 'num_block', 'enc_num_layer', 'dec_num_layer', 'fb_num_layer', 'enc_num_unit',
                    'dec_num_unit', 'fb_num_unit', 'q_train', 'q_train_fb', 'q_test_start', 'q_test_end',
                    'q_points', 'channel_mode', 'enc_act', 'zero_padding', 'file', 'BER_Par', 'BER_Impar', 'Model_Description']
    csv_file_path = '/content/datos.csv'
    model_description = str(model)
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=column_names)
        writer.writeheader()
        args_dict = vars(args)
        writer.writerow(args_dict)
        writer.writerow({'Model_Description': model_description})
    csv_df = pd.read_csv(csv_file_path)
    # Crear DataFrame con BER_PAR y BER_Impar
    data = {'Probabilidad Q': Q, 'BER': ber}
    ber_df = pd.DataFrame(data)
    # Ruta del archivo Excel
    excel_file_path = '/content/datos_con_args.xlsx'
    # Guardar el DataFrame de BER en un archivo Excel
    ber_df.T.to_excel(excel_file_path, index=True)
    # Leer el archivo Excel en un DataFrame
    excel_df = pd.read_excel(excel_file_path, index_col=0)
    # Concatenar los DataFrames
    df_concatenado = pd.concat([csv_df.T, excel_df])
    # Ruta del archivo concatenado
    concatenated_file_path = '/content/Resultados.xlsx'
    # Guardar el DataFrame concatenado en un archivo Excel
    df_concatenado.to_excel(concatenated_file_path, index=True)
    # Descargar el archivo concatenado
    files.download(concatenated_file_path)
    return concatenated_file_path

identity = str(np.random.random())[2:8]
print('[ID]', identity)

use_cuda = not args.no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
if use_cuda:
    model = AE(args).to(device)
else:
    model = AE(args)

print(model)

if args.init_nw_weight == 'default':
    pass
else:
    model = torch.load(args.init_nw_weight)
    model.args = args

# Guardar los parámetros iniciales del modelo
#initial_state_dict = model.state_dict()
#torch.save(initial_state_dict, 'initial_model2.pth')

initial_state_dict = torch.load('initial_model2.pth')
model.load_state_dict(initial_state_dict)

optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=args.learning_rate)
test_ratio = 20
num_train_block, num_test_block = args.num_block, args.num_block/test_ratio
my_q_train = args.q_train
my_q_train_fb = args.q_train_fb

print('Traning Q is', my_q_train)

################
#              #
#  ███████████ #
# ░█░░░███░░░█ #
# ░   ░███  ░  #
#     ░███     #
#     ░███     #
#     ░███     #
#     █████    #
#    ░░░░░     #
#              #
################

def train(epoch):
    model.train()
    train_loss = 0

    for batch_idx in range(int(num_train_block/args.batch_size)):
        if args.zero_padding:
            X_train    = torch.randint(0, 2, (args.batch_size, args.block_len, 1), dtype=torch.float)
            X_train    = torch.cat([X_train, torch.zeros(args.batch_size, 1, 1)], dim=1)
            fwd_noise =  torch.bernoulli(torch.full((args.batch_size, args.block_len+1, args.code_rate), my_q_train))
            fb_noise =   torch.bernoulli(torch.full((args.batch_size, args.block_len+1, args.code_rate), my_q_train_fb))
        else:
            X_train    = torch.randint(0, 2, (args.batch_size, args.block_len, 1), dtype=torch.float)
            fwd_noise =  torch.bernoulli(torch.full((args.batch_size, args.block_len, args.code_rate), my_q_train))
            fb_noise =   torch.bernoulli(torch.full((args.batch_size, args.block_len, args.code_rate), my_q_train_fb))

        X_train, fwd_noise, fb_noise = X_train.to(device), fwd_noise.to(device), fb_noise.to(device)
        optimizer.zero_grad()
        output, X_hat_train_code = model(X_train, fwd_noise, fb_noise)
        loss = F.binary_cross_entropy(output, X_train)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 10000 == 0:
            print('Train Epoch: {} [{}/{} Loss: {:.6f}'.format(epoch, batch_idx, num_train_block/args.batch_size, loss.item()))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss /(num_train_block/args.batch_size)) )
def test(qs):
  model.eval()
  torch.manual_seed(random.randint(0,1000))
  print('Qs', qs)
  num_train_block =  args.num_block
  errorxbit_T = torch.zeros(1, args.block_len, 1).to(device)
  promedio_errorxbit = torch.zeros(1, args.block_len, 1).to(device)
  BER_FINAL = []
  for q in qs:
    test_ber, test_bler = .0, .0
    with torch.no_grad():
      num_test_batch = int(num_train_block / (args.batch_size * test_ratio))
      for batch_idx in range(num_test_batch):
        if args.zero_padding:
          X_test = torch.randint(0, 2, (args.batch_size, args.block_len, 1), dtype=torch.float)
          X_test = torch.cat([X_test, torch.zeros(args.batch_size, 1, 1)], dim=1)
          fwd_noise = torch.bernoulli(torch.full((args.batch_size, args.block_len + 1, args.code_rate), q))
          fb_noise = torch.zeros((args.batch_size, args.block_len + 1, args.code_rate))

        else:
          X_test = torch.randint(0, 2, (args.batch_size, args.block_len, 1), dtype=torch.float)
          fwd_noise =  torch.bernoulli(torch.full((args.batch_size, args.block_len, args.code_rate), q))
          fb_noise =   torch.bernoulli(torch.full((args.batch_size, args.block_len, args.code_rate), my_q_train_fb))

        X_test, fwd_noise, fb_noise = X_test.to(device), fwd_noise.to(device), fb_noise.to(device)
        X_hat_test, X_hat_test_code = model(X_test, fwd_noise, fb_noise)
        X_hat_test.to(device)
        X_hat_test_code.to(device)
        test_ber += errors_ber(X_hat_test, X_test)
        test_bler += errors_bler(X_hat_test, X_test)
        promedio_errorxbit += ber_pos(X_hat_test,X_test)

    promedio_errorxbit /= 1.0*num_test_batch
    errorxbit_T = torch.add(errorxbit_T, promedio_errorxbit)
    test_ber  /= 1.0*num_test_batch
    test_bler /= 1.0*num_test_batch

    print('Test Q',round(q,3) ,'with ber ', float(test_ber), 'with bler', float(test_bler))
    BER_FINAL.append(q)
    BER_FINAL.append(float(test_ber))

  return errorxbit_T, BER_FINAL

#PATH='torch_model_791480.pt'
#model=torch.load(PATH)

q_interval = (args.q_test_end - args.q_test_start)* 1.0 /  (args.q_points)
qs = [q_interval* item + args.q_test_start for item in range(args.q_points)]

history_errorxbit_Ta = torch.zeros(1, args.block_len, 1).to(device)

for epoch in tqdm(range(1, args.num_epoch + 1)):
    print(" ")
    train(epoch)
    errorxbit_Ta, BER_FINAL = test(qs)

    if epoch >= int((args.num_epoch + 1)*0.9):
      history_errorxbit_Ta = torch.add(history_errorxbit_Ta, errorxbit_Ta)
    else:
      pass


errorxbit_T_q = errorxbit_Ta/(len(qs))

history_errorxbit_Tq = history_errorxbit_Ta/((len(qs))*((args.num_epoch + 1)-int((args.num_epoch + 1)*0.9)))


#lista_errxbit_q = errorxbit_T_q.cpu().squeeze().squeeze().numpy()
#lista_errxbit_q = errorxbit_T_q .squeeze().squeeze().numpy().to(device)

history_errorxbit_Tq = history_errorxbit_Tq.cpu().squeeze().squeeze().numpy()

print(BER_FINAL)
lista = list(range(1, (args.block_len+1)))


Guardar_resultados(BER_FINAL, args, model)
#plt.plot(lista, lista_errxbit_q, marker='o', linestyle=':')
plt.plot(lista, history_errorxbit_Tq, marker='o', linestyle=':')
plt.xlabel('Posición de bit')
plt.ylabel('BER')
plt.title('BER del bit bk')
plt.savefig('grafico.png')  # Guarda la figura como un archivo PNG
plt.show()
files.download('grafico.png')
print("                                                                       ")
print("                                          ███████╗    ██╗    ███╗   ██╗")
print("                                          ██╔════╝    ██║    ████╗  ██║")
print("                                          █████╗      ██║    ██╔██╗ ██║")
print("                                          ██╔══╝      ██║    ██║╚██╗██║")
print("                                          ██║         ██║    ██║ ╚████║")
print("                                          ╚═╝         ╚═╝    ╚═╝  ╚═══╝")
print("                                                                       ")
print(model)
#import pdb; pdb.set_trace()
#torch.save(model, '/content/drive/MyDrive/Deepcode_pytorch/tmp/torch_model_'+identity+'.pt')

#print('saved model', '/content/drive/MyDrive/Deepcode_pytorch/tmp/torch_model_'+identity+'.pt')



In [ ]:
#@title Deepcode: 2 Capas lineales Encoder 1 entradas, salidas 2 -> 2 Capas Lineales Decoder 2 entradas, 1 salida sin FEEDBACK, rate 1/2
__author__ = 'yihanjiang'
# Adding the *Receiver Encoding*
import argparse
import math
import random
import torch
import torch.optim as optim
import numpy as np
import torch.nn.functional as F
from tqdm import tqdm
import matplotlib.pyplot as plt
from google.colab import files
import csv
import locale
import pandas as pd

def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('-init_nw_weight', type=str, default='default')
    parser.add_argument('-code_rate', type=int, default=2)
    parser.add_argument('-learning_rate', type = float, default=0.00085)
    parser.add_argument('-batch_size', type=int, default=100)
    parser.add_argument('-num_epoch', type=int, default=10)
    parser.add_argument('--no-cuda', action='store_true', default=False, help='disables CUDA training')
    parser.add_argument('-block_len', type=int, default=50)
    parser.add_argument('-num_block', type=int, default=100000)
    parser.add_argument('-enc_num_layer', type=int, default=2)
    parser.add_argument('-dec_num_layer', type=int, default=2)
    parser.add_argument('-fb_num_layer',  type=int, default=2)
    parser.add_argument('-enc_num_unit',  type=int, default=100)
    parser.add_argument('-dec_num_unit',  type=int, default=100)
    parser.add_argument('-fb_num_unit',   type=int, default=100)
    parser.add_argument('-q_train', type=float, default=0.15, help='Valor de probabilidad q de entrenamiento')
    parser.add_argument('-q_train_fb', type=float, default=0.0000001, help='Valor de probabilidad q de entrenamiento para feedback')
    parser.add_argument('-q_test_start', type=float, default=0.001, help='Inicio de valores de q para pruebas')
    parser.add_argument('-q_test_end', type=float, default=0.15, help='Fin de valores de q para pruebas')
    parser.add_argument('-q_points', type=int, default=20, help='Número de puntos de probabilidad q')
    parser.add_argument('-channel_mode', choices=['normalize', 'lazy_normalize', 'tanh'], default='lazy_normalize')
    parser.add_argument('-enc_act', choices=['tanh', 'selu', 'relu', 'elu', 'sigmoid'], default='sigmoid')
    parser.add_argument('--zero_padding', action='store_true', default=False, help='enable zero padding')
    parser.add_argument("-f", "--file", required=False)
    args = parser.parse_args()
    return args
#############################
#                           #
#    █████████   ██████████ #
#   ███░░░░░███ ░░███░░░░░█ #
#  ░███    ░███  ░███  █ ░  #
#  ░███████████  ░██████    #
#  ░███░░░░░███  ░███░░█    #
#  ░███    ░███  ░███ ░   █ #
#  █████   █████ ██████████ #
# ░░░░░   ░░░░░ ░░░░░░░░░░  #
#                           #
#############################
class AE(torch.nn.Module):
    def __init__(self, args):
        super(AE, self).__init__()
        self.args             = args
        # Encoder
        self.enc_p2_linear_fwd   = torch.nn.Linear(1, args.enc_num_unit, bias=True)
        self.enc_p2_linear    = torch.nn.Linear(args.enc_num_unit, 2)

        #Decoder
        #self.dec_lstm           = torch.nn.LSTM(args.code_rate,  args.dec_num_unit,
        #                                   num_layers=2, bias=True, batch_first=True,
        #                                   dropout=0, bidirectional=True)
        self.dec_linear        = torch.nn.Linear(args.code_rate, args.dec_num_unit, bias=True)
        self.dec_output        = torch.nn.Linear(args.dec_num_unit, 1)

    def enc_act(self, inputs):
        if self.enc_act == 'tanh':
            return F.tanh(inputs)
        elif self.enc_act == 'elu':
            return F.elu(inputs)
        elif self.enc_act == 'relu':
            return F.relu(inputs)
        elif self.enc_act == 'selu':
            return F.selu(inputs)
        elif self.enc_act == 'sigmoid':
            return F.sigmoid(inputs)
        else:
            return F.tanh(inputs)

    def forward(self, input, fwd_noise, fb_noise):
        x_p1      = input
        x_p1_norm = 2*x_p1-1
        #p1_code = x_p1

        #if self.args.zero_padding:
        #    p1_rec  = torch.logical_xor(x_p1, fwd_noise[:,:,0].view(self.args.batch_size, self.args.block_len+1, 1) ).float()
        #    p1_fb   = torch.logical_xor(p1_rec , fb_noise[:,:, 0].view(self.args.batch_size, self.args.block_len+1, 1) ).float()
        #else:
        #    p1_rec  = torch.logical_xor(x_p1, fwd_noise[:,:,0].view(self.args.batch_size, self.args.block_len, 1) ).float()
        #    p1_fb   = torch.logical_xor(p1_rec , fb_noise[:,:, 0].view(self.args.batch_size, self.args.block_len, 1) ).float()

        #p1_fb_norm = 2*p1_fb - 1


        for idx in range(input.shape[1]):
            #if idx == 0:
            #    input_tmp        = x_p1_norm[:,idx,:].view(self.args.batch_size, 1, 1)
            #    x_fwd_p2  =        self.enc_act(self.enc_p2_linear_fwd(input_tmp))
            #    x_tmp_p2         = self.enc_act(self.enc_p2_linear(x_fwd_p2))
            #    x_p2_history     = x_tmp_p2

            #else:
            input_tmp        = x_p1_norm[:,idx,:].view(self.args.batch_size, 1, 1)
            x_tmp_p2  =        self.enc_act(self.enc_p2_linear_fwd(input_tmp))
            x_tmp_p2         = self.enc_act(self.enc_p2_linear(x_tmp_p2))
            x_p2_history     = x_tmp_p2

            x_tmp_p2  = torch.heaviside(x_tmp_p2 , torch.zeros(x_tmp_p2.shape, device=x_tmp_p2.device)).float()


            if idx == 0:
              x_tmp_p2_gen= x_tmp_p2

            else:
              x_tmp_p2_gen = torch.cat([x_tmp_p2_gen,x_tmp_p2 ], dim = 1)


            #if idx == 0:
            #    p2_code= x_tmp_p2

            #else:
            #    p2_code = torch.cat([p2_code,x_tmp_p2 ], dim = 1)


            x_p2_rec    = torch.logical_xor(x_tmp_p2,  fwd_noise[:,idx,:].view(self.args.batch_size, 1, 2)).float()
            #x_p2_fb     = torch.logical_xor(x_p2_rec, fb_noise[:,idx, 1:].view(self.args.batch_size, 1, 2)).float()

            #fb_tmp      = 2*x_p2_fb-1

            if idx == 0:
                #p2_code= x_tmp_p2
                p2_rec = x_p2_rec
                #p2_fb  = x_p2_fb
            else:
                #p2_code = torch.cat([p2_code,x_tmp_p2 ], dim = 1)
                p2_rec = torch.cat([p2_rec,x_p2_rec ], dim = 1)
                #p2_fb  = torch.cat([p2_fb, x_p2_fb],   dim = 1)

        dec_input = p2_rec

        x_code = x_tmp_p2_gen
        inputs_dec = 2*dec_input-1

        #x_dec, hdec_tmp  = self.dec_lstm( inputs_dec.float() )
        x_dec  = self.dec_linear( inputs_dec.float() )
        x_dec     = F.sigmoid(self.dec_output(x_dec))
        return x_dec, x_code
####################
#                  #
#  ██████   ██████ #
# ░░██████ ██████  #
#  ░███░█████░███  #
#  ░███░░███ ░███  #
#  ░███ ░░░  ░███  #
#  ░███      ░███  #
#  █████     █████ #
# ░░░░░     ░░░░░  #
#                  #
####################

args = get_args()
print(args)
def errors_ber(y_true, y_pred):
    if args.zero_padding:
        t1 = torch.round(y_true[:,:-1,:])
        t2 = torch.round(y_pred[:,:-1,:])
    else:
        t1 = torch.round(y_true[:,:,:])
        t2 = torch.round(y_pred[:,:,:])

    myOtherTensor = torch.not_equal(t1, t2).float()
    k = sum(sum(myOtherTensor))/(myOtherTensor.shape[0]*myOtherTensor.shape[1])
    return k
def errors_bler(y_true, y_pred):

    if args.zero_padding:
        t1 = torch.round(y_true[:,:-1,:])
        t2 = torch.round(y_pred[:,:-1,:])
    else:
        t1 = torch.round(y_true[:,:,:])
        t2 = torch.round(y_pred[:,:,:])

    decoded_bits = t1
    X_test       = t2
    tp0 = (abs(decoded_bits-X_test)).reshape([X_test.shape[0],X_test.shape[1]])

    bler_err_rate = sum(torch.sum(tp0,axis=1)>0)*1.0/(X_test.shape[0])
    return bler_err_rate

def ber_pos(y_true, y_pred):
    if args.zero_padding:
        t1 = torch.round(y_true[:,:-1,:])
        t2 = torch.round(y_pred[:,:-1,:])
    else:
        t1 = torch.round(y_true[:,:,:])
        t2 = torch.round(y_pred[:,:,:])
    dif = torch.not_equal(t1, t2).float()
    suma_dif = (torch.sum(dif, dim=0))
    promedio_errorxbit = suma_dif/(dif.shape[0])
    return promedio_errorxbit

def Guardar_resultados(BER_FINAL, args, model):
    Q = [BER_FINAL[i] for i in range(len(BER_FINAL)) if i % 2 == 0]
    ber = [BER_FINAL[i] for i in range(len(BER_FINAL)) if i % 2 != 0]
    column_names = ['init_nw_weight', 'code_rate', 'learning_rate', 'batch_size', 'num_epoch', 'no_cuda',
                    'block_len', 'num_block', 'enc_num_layer', 'dec_num_layer', 'fb_num_layer', 'enc_num_unit',
                    'dec_num_unit', 'fb_num_unit', 'q_train', 'q_train_fb', 'q_test_start', 'q_test_end',
                    'q_points', 'channel_mode', 'enc_act', 'zero_padding', 'file', 'BER_Par', 'BER_Impar', 'Model_Description']
    csv_file_path = '/content/datos.csv'
    model_description = str(model)
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=column_names)
        writer.writeheader()
        args_dict = vars(args)
        writer.writerow(args_dict)
        writer.writerow({'Model_Description': model_description})
    csv_df = pd.read_csv(csv_file_path)
    # Crear DataFrame con BER_PAR y BER_Impar
    data = {'Probabilidad Q': Q, 'BER': ber}
    ber_df = pd.DataFrame(data)
    # Ruta del archivo Excel
    excel_file_path = '/content/datos_con_args.xlsx'
    # Guardar el DataFrame de BER en un archivo Excel
    ber_df.T.to_excel(excel_file_path, index=True)
    # Leer el archivo Excel en un DataFrame
    excel_df = pd.read_excel(excel_file_path, index_col=0)
    # Concatenar los DataFrames
    df_concatenado = pd.concat([csv_df.T, excel_df])
    # Ruta del archivo concatenado
    concatenated_file_path = '/content/Resultados.xlsx'
    # Guardar el DataFrame concatenado en un archivo Excel
    df_concatenado.to_excel(concatenated_file_path, index=True)
    # Descargar el archivo concatenado
    files.download(concatenated_file_path)
    return concatenated_file_path

identity = str(np.random.random())[2:8]
print('[ID]', identity)

use_cuda = not args.no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
if use_cuda:
    model = AE(args).to(device)
else:
    model = AE(args)

print(model)

if args.init_nw_weight == 'default':
    pass
else:
    model = torch.load(args.init_nw_weight)
    model.args = args

optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=args.learning_rate)
test_ratio = 1
num_train_block, num_test_block = args.num_block, args.num_block/test_ratio
my_q_train = args.q_train
my_q_train_fb = args.q_train_fb

print('Traning Q is', my_q_train)

################
#              #
#  ███████████ #
# ░█░░░███░░░█ #
# ░   ░███  ░  #
#     ░███     #
#     ░███     #
#     ░███     #
#     █████    #
#    ░░░░░     #
#              #
################

def train(epoch):
    model.train()
    train_loss = 0

    for batch_idx in range(int(num_train_block/args.batch_size)):
        if args.zero_padding:
            X_train    = torch.randint(0, 2, (args.batch_size, args.block_len, 1), dtype=torch.float)
            X_train    = torch.cat([X_train, torch.zeros(args.batch_size, 1, 1)], dim=1)
            fwd_noise =  torch.bernoulli(torch.full((args.batch_size, args.block_len+1, args.code_rate), my_q_train))
            fb_noise =   torch.bernoulli(torch.full((args.batch_size, args.block_len+1, args.code_rate), my_q_train_fb))
        else:
            X_train    = torch.randint(0, 2, (args.batch_size, args.block_len, 1), dtype=torch.float)
            fwd_noise =  torch.bernoulli(torch.full((args.batch_size, args.block_len, args.code_rate), my_q_train))
            fb_noise =   torch.bernoulli(torch.full((args.batch_size, args.block_len, args.code_rate), my_q_train_fb))

        X_train, fwd_noise, fb_noise = X_train.to(device), fwd_noise.to(device), fb_noise.to(device)
        optimizer.zero_grad()
        output, X_hat_train_code = model(X_train, fwd_noise, fb_noise)
        loss = F.binary_cross_entropy(output, X_train)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 10000 == 0:
            print('Train Epoch: {} [{}/{} Loss: {:.6f}'.format(epoch, batch_idx, num_train_block/args.batch_size, loss.item()))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss /(num_train_block/args.batch_size)) )
def test(qs):
  model.eval()
  torch.manual_seed(random.randint(0,1000))
  print('Qs', qs)
  num_train_block =  args.num_block
  errorxbit_T = torch.zeros(1, args.block_len, 1).to(device)
  promedio_errorxbit = torch.zeros(1, args.block_len, 1).to(device)
  BER_FINAL = []
  for q in qs:
    test_ber, test_bler = .0, .0
    with torch.no_grad():
      num_test_batch = int(num_train_block / (args.batch_size * test_ratio))
      for batch_idx in range(num_test_batch):
        if args.zero_padding:
          X_test = torch.randint(0, 2, (args.batch_size, args.block_len, 1), dtype=torch.float)
          X_test = torch.cat([X_test, torch.zeros(args.batch_size, 1, 1)], dim=1)
          fwd_noise = torch.bernoulli(torch.full((args.batch_size, args.block_len + 1, args.code_rate), q))
          fb_noise = torch.zeros((args.batch_size, args.block_len + 1, args.code_rate))

        else:
          X_test = torch.randint(0, 2, (args.batch_size, args.block_len, 1), dtype=torch.float)
          fwd_noise =  torch.bernoulli(torch.full((args.batch_size, args.block_len, args.code_rate), q))
          fb_noise =   torch.bernoulli(torch.full((args.batch_size, args.block_len, args.code_rate), my_q_train_fb))

      X_test, fwd_noise, fb_noise = X_test.to(device), fwd_noise.to(device), fb_noise.to(device)
      X_hat_test, X_hat_test_code = model(X_test, fwd_noise, fb_noise)
      X_hat_test.to(device)
      X_hat_test_code.to(device)
      test_ber += errors_ber(X_hat_test, X_test)
      test_bler += errors_bler(X_hat_test, X_test)
      promedio_errorxbit += ber_pos(X_hat_test,X_test)

    promedio_errorxbit /= 1.0*num_test_batch
    errorxbit_T = torch.add(errorxbit_T, promedio_errorxbit)
    test_ber  /= 1.0*num_test_batch
    test_bler /= 1.0*num_test_batch

    print('Test Q',round(q,3) ,'with ber ', float(test_ber), 'with bler', float(test_bler))
    BER_FINAL.append(q)
    BER_FINAL.append(float(test_ber))

  return errorxbit_T, BER_FINAL

#PATH='torch_model_791480.pt'
#model=torch.load(PATH)

q_interval = (args.q_test_end - args.q_test_start)* 1.0 /  (args.q_points)
qs = [q_interval* item + args.q_test_start for item in range(args.q_points)]

history_errorxbit_Ta = torch.zeros(1, args.block_len, 1).to(device)

for epoch in tqdm(range(1, args.num_epoch + 1)):
    print(" ")
    train(epoch)
    errorxbit_Ta, BER_FINAL = test(qs)

    if epoch >= int((args.num_epoch + 1)*0.9):
      history_errorxbit_Ta = torch.add(history_errorxbit_Ta, errorxbit_Ta)
    else:
      pass


errorxbit_T_q = errorxbit_Ta/(len(qs))

history_errorxbit_Tq = history_errorxbit_Ta/((len(qs))*((args.num_epoch + 1)-int((args.num_epoch + 1)*0.9)))


#lista_errxbit_q = errorxbit_T_q.cpu().squeeze().squeeze().numpy()
#lista_errxbit_q = errorxbit_T_q .squeeze().squeeze().numpy().to(device)

history_errorxbit_Tq = history_errorxbit_Tq.cpu().squeeze().squeeze().numpy()

print(BER_FINAL)
lista = list(range(1, (args.block_len+1)))


Guardar_resultados(BER_FINAL, args, model)
#plt.plot(lista, lista_errxbit_q, marker='o', linestyle=':')
plt.plot(lista, history_errorxbit_Tq, marker='o', linestyle=':')
plt.xlabel('Posición de bit')
plt.ylabel('BER')
plt.title('BER del bit bk')
plt.savefig('grafico.png')  # Guarda la figura como un archivo PNG
plt.show()
files.download('grafico.png')
print("                                                                       ")
print("                                          ███████╗    ██╗    ███╗   ██╗")
print("                                          ██╔════╝    ██║    ████╗  ██║")
print("                                          █████╗      ██║    ██╔██╗ ██║")
print("                                          ██╔══╝      ██║    ██║╚██╗██║")
print("                                          ██║         ██║    ██║ ╚████║")
print("                                          ╚═╝         ╚═╝    ╚═╝  ╚═══╝")
print("                                                                       ")
print(model)
#import pdb; pdb.set_trace()
#torch.save(model, '/content/drive/MyDrive/Deepcode_pytorch/tmp/torch_model_'+identity+'.pt')

#print('saved model', '/content/drive/MyDrive/Deepcode_pytorch/tmp/torch_model_'+identity+'.pt')

